In [2]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from data import Data, MATLAB_Data
from loss import Loss
from model import ComplexModel
from model.common import CPLX
from trainer import Trainer
from radar import DualRadar, DualRadarSAR
from saved import Saver

In [ ]:
def test_net(input, net_path="./saved/fftnet5.tar"):
    """Applies dual-radar-SR network to data from MATLAB

    Args:
        input (MATLAB): Nk x N complex array from MATLAB, where N is the number of samples
    """
    
    input = np.array(input).transpose()
    N, Nk = input.shape
    input = input.reshape((N, 1, Nk))
    input = torch.from_numpy(input)
    
    dr = DualRadar()
    d = Data(dr, 0)
    
    s = Saver()
    args, m, l, t = s.Load(d, ComplexModel, Loss, Trainer, net_path)
    d = Data(dr, args)
    d.create_dataset_train(1, 1, "xz")
    
    input = input.to(args.device)
    x_in = CPLX(torch.real(input), torch.imag(input))
    Y_pred = m.model(x_in)
    output = (Y_pred.r + 1j*Y_pred.i).cpu().detach().numpy().reshape((N, Nk))
    
    return output
    
    
    
    
    
    